In [205]:
# Ziwei Qiu, ziweiqiu@g.harvard.edu
from dimod import DiscreteQuadraticModel
from dimod import ExactSolver
import pandas as pd
import sys
from dwave.system import LeapHybridDQMSampler
from neal import SimulatedAnnealingSampler
from math import log2, floor
import dimod
import os
import numpy as np

## Problem Definition

In [202]:
num_of_items = 5
values = list(np.random.randint(1,5, size=(num_of_items)))
weights = list(np.random.randint(1,5, size=(num_of_items)))
bound = list(np.random.randint(12, 20,size=(num_of_items))+2)
# weights = [1,4]
# values = [3,4]
# bound = [10,10]
bound = [b+1 for b in bound] # take into account 0
weight_limit = 10

print('values:',values)
print('weights:',weights)
print('bound:',bound)
print('weight_limit:',weight_limit)
# Lagrange multipliers A>max(costs)B>0
# B = 1
A = max(values)*3

values: [4, 2, 2, 2, 2]
weights: [3, 1, 3, 1, 3]
bound: [18, 18, 19, 17, 17]
weight_limit: 10


## Create the DQM model


In [203]:
##@  Discrete Quadratic Model @##
dqm = DiscreteQuadraticModel()

x = []
#@ Add variables @##
for k in range(num_of_items):
    x.append(dqm.add_variable(bound[k], label='x' + str(k))) # number of discrete values 

for n in range(1,weight_limit+1):
    dqm.add_variable(2, label='y' + str(n)) # either 0 or 1, 2 values possible
    
##@ Hamiltonian xi-xi terms ##
for k in range(num_of_items):
    pieces = range(bound[k])
#     dqm.set_linear('x' + str(k),  - values[k]*pieces)
    dqm.set_linear('x' + str(k), A * (weights[k]**2) * (np.array(pieces)**2) - values[k]*np.array(pieces))

# Hamiltonian y-y terms
for n in range(1,weight_limit+1):
    dqm.set_linear('y' + str(n), A*np.array([0,1])* (n**2-1))
    
# Hamiltonian yi-yj terms 
for n in range(1,weight_limit+1):
    for m in range(n + 1, weight_limit+1): 
        dqm.set_quadratic('y' + str(n), 'y' + str(m), {(1,1):2 * A * (1+m*n)})
        
# # Hamiltonian xi-xj terms
for i in range(num_of_items):
    for j in range(i + 1, num_of_items):
        biases_dict = {}
        for piece1 in range(bound[i]):
            for piece2 in range(bound[j]):
                biases_dict[(piece1, piece2)]=(2 * A * weights[i] * weights[j])*piece1*piece2
        dqm.set_quadratic('x' + str(i), 'x' + str(j), biases_dict)
    
# Hamiltonian x-y terms
for i in range(num_of_items):
    for n in range(1,weight_limit+1):
        biases_dict = {}
        for piece1 in range(bound[i]):
            biases_dict[(piece1, 1)]=-2 * A * weights[i] * n* piece1

        dqm.set_quadratic('x' + str(i), 'y' + str(n), biases_dict) 

In [225]:
sampler = LeapHybridDQMSampler()
response = sampler.sample_dqm(dqm)
sample = response.first.sample
energy = response.first.energy
best_solution = response.first.sample
best_solution = [best_solution[i] for i in x]
total_weights = sum([weights[i]*best_solution[i] for i in range(len(x))])
total_value = sum([values[i]*best_solution[i] for i in range(len(x))])
print(best_solution)
print('Total weight:',total_weights)
print('Total value:',total_value)